In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U xformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U einops
!pip install -q -U nvidia-ml-py3
!pip install -q -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from pynvml import *
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import time, torch

In [ ]:

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

In [ ]:
base_model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import accelerate
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map = {"": 0}, torch_dtype="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [ ]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj"])

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PhiForCausalLM(
          (model): PhiModel(
            (embed_tokens): Embedding(51200, 2560)
            (embed_dropout): Dropout(p=0.0, inplace=False)
            (layers): ModuleList(
              (0-31): 32 x PhiDecoderLayer(
                (self_attn): PhiAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2560, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=2560, bias=False)
                    )
                  

In [ ]:
dataset = load_dataset("csv", data_files="prompts.csv")

In [ ]:
def create_prompt(sample):
  system_prompt_template = """<s>
Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :<<user_question>>
### Database Schema:
<<database_schema>>
### Response:
<<user_response>>
</s>
"""
  user_message = sample['question']
  user_response = sample['answer']
  database_schema = sample['context']
  prompt_template = system_prompt_template.replace("<<user_question>>",f"{user_message}").replace("<<user_response>>",f"{user_response}").replace("<<database_schema>>",f"{database_schema} ")

  return {"inputs":prompt_template}

#
instruct_tune_dataset = dataset.map(create_prompt)
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context', 'inputs'],
        num_rows: 113
    })
})


In [ ]:
training_arguments = TrainingArguments(
        output_dir="phi2-results2",
        save_strategy="epoch",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=12,
        log_level="debug",
        save_steps=100,
        logging_steps=25,
        learning_rate=1e-4,
        eval_steps=50,
        optim='paged_adamw_8bit',
        fp16=True,
        num_train_epochs=1,
        max_steps=200,
        warmup_steps=10,
        lr_scheduler_type="linear",
        seed=42,)

In [ ]:
train_dataset = instruct_tune_dataset.map(batched=True,remove_columns=['answer', 'question', 'context'])
train_dataset

DatasetDict({
    train: Dataset({
        features: ['inputs'],
        num_rows: 113
    })
})

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset["train"],
        #eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="inputs",
        max_seq_length=1024,
        tokenizer=tokenizer,
        args=training_arguments,
        packing=False
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 4
***** Running training *****
  Num examples = 113
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 12
  Total optimization steps = 200
  Number of trainable parameters = 5,242,880


Step,Training Loss
25,1.664400
50,0.949700
75,0.504600
100,0.387400
125,0.336200
150,0.313400
175,0.297600
200,0.288100


Saving model checkpoint to phi2-results2/tmp-checkpoint-2
tokenizer config file saved in phi2-results2/tmp-checkpoint-2/tokenizer_config.json
Special tokens file saved in phi2-results2/tmp-checkpoint-2/special_tokens_map.json
Saving model checkpoint to phi2-results2/tmp-checkpoint-4
tokenizer config file saved in phi2-results2/tmp-checkpoint-4/tokenizer_config.json
Special tokens file saved in phi2-results2/tmp-checkpoint-4/special_tokens_map.json
Saving model checkpoint to phi2-results2/tmp-checkpoint-7
tokenizer config file saved in phi2-results2/tmp-checkpoint-7/tokenizer_config.json
Special tokens file saved in phi2-results2/tmp-checkpoint-7/special_tokens_map.json
Saving model checkpoint to phi2-results2/tmp-checkpoint-9
tokenizer config file saved in phi2-results2/tmp-checkpoint-9/tokenizer_config.json
Special tokens file saved in phi2-results2/tmp-checkpoint-9/special_tokens_map.json
Saving model checkpoint to phi2-results2/tmp-checkpoint-12
tokenizer config file saved in phi2-r

TrainOutput(global_step=200, training_loss=0.5926899790763855, metrics={'train_runtime': 3900.2421, 'train_samples_per_second': 2.461, 'train_steps_per_second': 0.051, 'total_flos': 2.805847879836672e+16, 'train_loss': 0.5926899790763855, 'epoch': 82.76})

In [ ]:
p = train_dataset['train'][0]['inputs']
p

'<s>\nBelow is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction :List all products along with their prices.\n### Database Schema:\nProducts (product_id, name, price, stock_quantity) \n### Response:\nSELECT name, price FROM Products;\n</s>\n'

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction :List all products along with their prices.\n### Database Schema:\nProducts (product_id, name, price, stock_quantity) \n### Response:\n"

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=200,)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction :List all products along with their prices.
### Database Schema:
Products (product_id, name, price, stock_quantity) 
### Response:
SELECT name, price FROM Products;



In [ ]:
trainer.save_model()

Saving model checkpoint to phi2-results2
tokenizer config file saved in phi2-results2/tokenizer_config.json
Special tokens file saved in phi2-results2/special_tokens_map.json


In [ ]:
p = train_dataset['train'][8]['inputs']
p

"<s>\nBelow is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction :Show the orders placed on 13-06-2023.\n### Database Schema:\nOrders (order_id, customer_id, order_date) \n### Response:\nSELECT * FROM Orders WHERE order_date = '13-06-2023';\n</s>\n"

In [ ]:
prompt = "Below is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction :Show the orders placed on 13-06-2023.\n### Database Schema:\nOrders (order_id, customer_id, order_date) \n### Response:\n"

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=200,)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :Show the orders placed on 13-06-2023.
### Database Schema:
Orders (order_id, customer_id, order_date) 
### Response:
SELECT * FROM Orders WHERE order_date = '13-06-2023';



In [ ]:
prompt = "Below is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction :List all products along with their prices and quantities in stock, sorted by their names alphabetically.\n### Database Schema:\nProducts (product_id, name, price, stock_quantity) \n### Response:\n"

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=200,)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction :List all products along with their prices and quantities in stock, sorted by their names alphabetically.
### Database Schema:
Products (product_id, name, price, stock_quantity) 
### Response:
SELECT name, price, stock_quantity FROM Products ORDER BY name;



In [ ]:
prompt = "Below is an instruction that describes a task.Write a response that appropriately completes the request.\n### Instruction : Display the products with the highest stock quantities.\n### Database Schema:\n Products (product_id, name, price, stock_quantity) \n### Response:\n"

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=200,)
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task.Write a response that appropriately completes the request.
### Instruction : Display the products with the highest stock quantities.
### Database Schema:
 Products (product_id, name, price, stock_quantity) 
### Response:
SELECT * FROM Products WHERE stock_quantity = (SELECT MAX(stock_quantity) FROM Products);

